## Using KerasNeuralFinder

In [1]:
#Importing the necessary libraries
import tensorflow as tf
from tensorflow import keras
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import shapiro
import time
import datetime
from sklearn.preprocessing import MinMaxScaler
import IPython
import os
import math
%matplotlib inline

from sklearn.preprocessing import MinMaxScaler
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras import optimizers
from tensorflow.keras import regularizers
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score, precision_recall_curve, auc

In [2]:
import KerasOptimizerNN
from KerasOptimizerNN import KerasNeuralFinder

In [3]:
X_temp = pd.read_csv("X_temp.csv").values
y_temp = pd.read_csv("y_temp.csv").values

In [4]:
X_train = pd.read_csv("X_train.csv").values
y_train = pd.read_csv("y_train.csv").values

In [5]:
y_temp = y_temp[:,-1]
y_train = y_train[:,-1]

In [6]:
param_grid = {}
param_grid['hidden_layer_neurons'] =[ [11, 8, 6], [8,5]]
param_grid['output_layer_neurons'] = [1]
param_grid['hidden_layer_activations'] = ['sigmoid']
param_grid['output_layer_activations'] = ['sigmoid']
param_grid['kernel_inializers'] = ['glorot_normal']
param_grid['bias_initializers'] = ['glorot_normal']
param_grid['optimizers'] = [ 'Adam']
param_grid['epochs'] = [20]
param_grid['learning_rate'] = [0.01]
param_grid['loss_functions'] = ['binary_crossentropy']

### New stuff!!!

* Can provide different additional metrics (built-in only at the moment) like binary_accuracy
* Mean and the min/max based on the confidence interval provided by user are returned in results
* get_best_results() method implemented get the best out of the run. Example usage below.
* Can choose to retain weights for future re-run using flag - retain_weights
* Can choose to reuse already stored weights to get same results

#### Roadmap

* To be able to store weights for more than 1 historical run
* Then we shall be able to choose the best weights as well, along with the best neural network design
* To implement parallel processing using multiprocessing library


In [7]:
knf = KerasNeuralFinder()
results = knf.fit(param_grid=param_grid, store_results=True, silent_mode=True,
                  X=X_temp, y=y_temp, fold=2,
                  confidence_interval=80,
                  metrics=['binary_accuracy', 'accuracy'])

Choice 2/2..., epoch = 20, time_taken=2.44 seconds, loss=49.57043379545212, accuracy=80.0000011920929
Parameters are :

{'hidden_layer_neurons': [8, 5]}
{'output_layer_neurons': 1}
{'hidden_layer_activations': 'sigmoid'}
{'output_layer_activations': 'sigmoid'}
{'kernel_inializers': 'glorot_normal'}
{'bias_initializers': 'glorot_normal'}
{'optimizers': 'Adam'}
{'epochs': 20}
{'learning_rate': 0.01}
{'loss_functions': 'binary_crossentropy'}

Completed running ... overall time taken = 5.4


In [8]:
results.T

,0,1
hidden_layer_neurons,"11,8,6","8,5"
output_layer_neurons,1,1
hidden_layer_activations,sigmoid,sigmoid
output_layer_activations,sigmoid,sigmoid
kernel_inializers,glorot_normal,glorot_normal
bias_initializers,glorot_normal,glorot_normal
optimizers,Adam,Adam
epochs,20,20
learning_rate,0.01,0.01
loss_functions,binary_crossentropy,binary_crossentropy


In [9]:
knf.get_best_result(results, by={'val_loss': 'low'})

,hidden_layer_neurons,output_layer_neurons,hidden_layer_activations,output_layer_activations,kernel_inializers,bias_initializers,optimizers,epochs,learning_rate,loss_functions,...,max_accuracy(80%),val_loss,min_val_loss(80%),max_val_loss(80%),val_binary_accuracy,min_val_binary_accuracy(80%),max_val_binary_accuracy(80%),val_accuracy,min_val_accuracy(80%),max_val_accuracy(80%)
1,"8,5",1,sigmoid,sigmoid,glorot_normal,glorot_normal,Adam,20,0.01,binary_crossentropy,...,80.0,49.682778,49.26,50.1,80.000001,80.0,80.0,80.000001,80.0,80.0
